In [ ]:
# Convertir variables categoricas en variables dummies. OneHot Encoding
# Standarizar
# Evaluar como funcionan los modelos inicialmente, en caso de que no den buenos resultados:
    # Probar con balanceo de clase.
    # Discretizar variables numéricas continuas, generando intervalos se puede utilizar WoE.
    # Probar con ambos conjuntos de datos. 